In [2]:
# !pip install transformers
# !pip install adapter-transformers
from transformers import RobertaModel, list_adapters
from RoBERTa import *
import torch

custom_roberta = CustomRobertaModel.from_pretrained('roberta-base')
adapter_name = custom_roberta.load_adapter("AdapterHub/bert-base-uncased-pf-imdb", source="hf")

custom_roberta.active_adapters = adapter_name
for name, param in custom_roberta.named_parameters():
    if f"adapters.{adapter_name}" not in name:
        param.requires_grad = False



Some weights of the model checkpoint at roberta-base were not used when initializing CustomRobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing CustomRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [3]:
from transformers import RobertaTokenizer, RobertaModel

# Load the RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

model = RobertaModel.from_pretrained('roberta-base')

input_text = "This is an example sentence."
encoded_input = tokenizer(input_text, return_tensors='pt')

outputs = model(**encoded_input)
outputs1 = custom_roberta(**encoded_input)

# Get the CLS output (the first token embedding)
cls_output = outputs.last_hidden_state[:, 0, :]
cls_output1 = outputs1.last_hidden_state[:, 0, :]
print(cls_output[0,0])
print(cls_output1[0,0])

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


TypeError: RobertaModel.forward() got an unexpected keyword argument 'adapter_names'